## homework-01 (q1-q6)

docker+sql (csv / parquet data download, postgres upload, pandas sql)

### question 1: 

answer: pip v 25.3  

``` 
docker run -it --entrypoint bash python:3.13

pip --version
```

## question 2:
answer: db:5432, (container names also resolve as hostnames in compose networks, so postgres:5432 is also valid)

## questions 3-6 setup: 

spin up containers with: 

``` 
docker compose up -d 
``` 

install data in current dir:

```
wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet
```

```
wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
```

In [1]:
import pandas as pd
import pyarrow 
import tqdm
from sqlalchemy import create_engine
from sqlalchemy import text

In [13]:
pg_user = "postgres"
pg_pass = "postgres" 
pg_host = "localhost" 
pg_port = "5433" 
pg_db = "ny_taxi"

gt_table_name = "green_trip_taxi" 

tz_table_name = "taxi_zone_lookup"

engine = create_engine(
    f"postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}"
)

read taxi and lookup data, load into sql

In [3]:
green_trip = pd.read_parquet("./green_tripdata_2025-11.parquet") 
green_trip.to_sql(gt_table_name, con=engine, if_exists="replace")
green_trip.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [4]:
taxi_zone_lookup = pd.read_csv("./taxi_zone_lookup.csv")
taxi_zone_lookup.to_sql(tz_table_name, con=engine, if_exists="replace")
taxi_zone_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


## question 3: 

In [17]:
start = '2025-11-01' 
end = '2025-12-01'
max_distance = 1

with engine.connect() as conn:
  df = pd.read_sql(text("""
          SELECT COUNT(*) FROM green_trip_taxi WHERE
          lpep_pickup_datetime >= :start AND
          lpep_pickup_datetime < :end AND
          trip_distance <= :distance
          """), conn, params={'start': start, 'end': end, 'distance': max_distance})
    
df

,count
0,8007


answer: 8007

## question 4: 

In [18]:
with engine.connect() as conn:
  df = pd.read_sql(text("""
        SELECT DATE(lpep_pickup_datetime) as day, MAX(trip_distance) as max_distance
        FROM green_trip_taxi
        WHERE DATE(lpep_pickup_datetime) IN ('2025-11-14', '2025-11-20', '2025-11-23', '2025-11-25')
        AND trip_distance > 100
        GROUP BY DATE(lpep_pickup_datetime)
        ORDER BY day
        """), conn)
  

df

,day,max_distance
0,2025-11-20,26823.53


answer: 2025-11-20

## question 5: 

(could setup tables with fks to zone id's for q5-6, but seems overkill for this)

In [19]:
with engine.connect() as conn:
  df = pd.read_sql(text("""
          SELECT 
              tz."Zone",
              tz."LocationID",
              MAX(gt.total_amount) as max_total_amount
          FROM green_trip_taxi gt
          INNER JOIN taxi_zone_lookup tz ON gt."PULocationID" = tz."LocationID"
          WHERE DATE(gt.lpep_pickup_datetime) = '2025-11-18' 
          AND tz."Zone" IN ('East Harlem North', 'East Harlem South', 
                           'Morningside Heights', 'Forest Hills')
          GROUP BY tz."Zone", tz."LocationID"
          ORDER BY max_total_amount DESC
          """), conn)
df

,Zone,LocationID,max_total_amount
0,East Harlem South,75,104.64
1,Morningside Heights,166,78.78
2,East Harlem North,74,78.44
3,Forest Hills,95,63.84


answer: LocationID 75: East Harlem South

## question 6: 

In [20]:
with engine.connect() as conn:
  df = pd.read_sql(text("""
          SELECT 
              tz."Zone" as dropoff_zone,
              tz."LocationID" as DOLocationID,
              MAX(gt.tip_amount) as largest_tip
          FROM green_trip_taxi gt
          INNER JOIN taxi_zone_lookup tz ON gt."DOLocationID" = tz."LocationID"
          WHERE DATE(gt.lpep_pickup_datetime) >= '2025-11-01' 
          AND DATE(gt.lpep_pickup_datetime) < '2025-12-01'
          AND gt."PULocationID" = 74
          GROUP BY tz."Zone", tz."LocationID"
          ORDER BY largest_tip DESC
          """), conn)
df

,dropoff_zone,dolocationid,largest_tip
0,Yorkville West,263,81.89
1,LaGuardia Airport,138,50.00
2,East Harlem North,74,45.00
3,Long Island City/Queens Plaza,146,34.25
4,None,265,28.90
...,...,...,...
132,South Williamsburg,217,0.00
133,Van Cortlandt Village,241,0.00
134,Van Nest/Morris Park,242,0.00
135,West Village,249,0.00


answer: Yorkville West